In [8]:
import os
import joblib
import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifierCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    classification_report,
    confusion_matrix
)
import seaborn as sns
import matplotlib.pyplot as plt


In [9]:
# Load preprocessed CSVs
X_train = pd.read_csv("data/processed/X_train_res.csv")
y_train = pd.read_csv("data/processed/y_train_res.csv").squeeze()
X_test = pd.read_csv("data/processed/X_test.csv")
y_test = pd.read_csv("data/processed/y_test.csv").squeeze()

# Adjust labels if needed (e.g., convert from [1,2,3] → [0,1,2])
y_train = y_train - 1
y_test = y_test - 1

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print("Train class distribution:\n", y_train.value_counts())

rf_best = joblib.load("models/random_forest.joblib")
xgb_best = joblib.load("models/xgboost.joblib")

(3978, 21) (3978,) (426, 21) (426,)
Train class distribution:
 NSP
2.0    1326
0.0    1326
1.0    1326
Name: count, dtype: int64


C:\Users\shriv\AppData\Local\Programs\Python\Python312\Lib\pickle.py:1710: UserWarning: [19:41:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\data\../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [10]:
# Soft voting uses predicted probabilities → more stable for unbalanced data
voting_clf = VotingClassifier(
    estimators=[
        ('rf', rf_best),
        ('xgb', xgb_best)
    ],
    voting='soft'
)
voting_clf.fit(X_train, y_train)

y_pred_vote = voting_clf.predict(X_test)

AttributeError: 'XGBModel' object has no attribute 'feature_weights'

In [ ]:
stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_best),
        ('xgb', xgb_best)
    ],
    final_estimator=make_pipeline(
        StandardScaler(),
        LogisticRegression(max_iter=2000, solver='lbfgs')
    ),
    stack_method='predict_proba',
    passthrough=False
)

In [ ]:
def evaluate_model(name, y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    print(f"\n{name} Results:")
    print(f"Accuracy: {acc:.4f} | F1 (macro): {f1:.4f} | Recall (macro): {recall:.4f}")
    print(classification_report(y_true, y_pred, digits=3))
    return acc, f1, recall

metrics = {}
metrics['Voting Ensemble'] = evaluate_model("Voting Ensemble", y_test, y_pred_vote)
metrics['Stacking Ensemble'] = evaluate_model("Stacking Ensemble", y_test, y_pred_stack)

In [ ]:
metrics_df = pd.DataFrame(metrics, index=["Accuracy", "F1_macro", "Recall_macro"]).T
print("\nOverall Comparison:\n", metrics_df)

sns.barplot(data=metrics_df, x=metrics_df.index, y="F1_macro")
plt.title("Ensemble Model Comparison (F1 Macro)")
plt.show()

In [ ]:
os.makedirs("models", exist_ok=True)
joblib.dump(voting_clf, "models/best_voting_ensemble.joblib")
joblib.dump(stacking_clf, "models/best_stacking_ensemble.joblib")
print("\n Ensembles completed and models saved")